In [39]:
def loadData(path=''):
    
    f = open(path + 'house-votes-84.data')
    txt = f.read()
    f.close()
    lst_txt = txt.split('\n')

    data = []

    for txt_line in lst_txt:
        tmp = txt_line.split(',')
        data.append(tmp)

    return data


In [40]:
def preProcessing(data, vote_y_n):
    
    data_pre = []
    for data_line in data:
        tmp_data = []
        for i in range(1, len(data_line)):
            #从第二列开始，将数据文件中的记录与当前的选择vote_y_n进行比较，若找到了相关记录，把下标存进去
            if (data_line[i] == vote_y_n):
                tmp_data.append(i)

        if (tmp_data == []):
            continue#如果当前这一条记录中没有任何一个项是vote_y_n对应的选项，直接进行下一个记录的查找
        data_pre.append(tmp_data)

    return data_pre

In [41]:
def rule_mining(data, support, confidence):
   
    dic_1 = mining_first(data, support, confidence)
    
    dic_2 = mining_second(data, dic_1, support, confidence)
    dic_before = dic_2
    dic_r = []
    
    #循环直到不再有新的频繁项集产生为止
    while (dic_before != {}):
        dic_r.append(dic_before)
        dic_3 = mining_third(data, dic_before, support, confidence)
        dic_before = dic_3

    return dic_r
    pass


In [42]:
def mining_first(data, support, confidence): #挖掘候选1-项集
 
    dic = {}
    count = len(data)
    for data_line in data:
        #对于数据集中的每一行投票数据
        for data_item in data_line:
            #对于每一行数据中的下标（对应某个议题）
            if (data_item in dic):
                #以键值对的形式进行存储和计数
                dic[data_item] += 1
            else:
                dic[data_item] = 1

    assert (support >= 0) and (support <= 1), 'suport must be in 0-1'
    #依靠给定的支持度阈值和投票数据的总数的得到满足条件的最小支持度值
    val_suport = int(count * support)
    assert (confidence >= 0) and (confidence <= 1), 'coincidence must be in 0-1'
    #如果键值对中的值大于或等于当前支持度阈值，则可以将该键值对作为频繁1-项集保留
    dic_1 = {}
    for item in dic:#如果对每一个议题的所选定的（y|n）进行计数，若计数总值超过了支持度所需要的计数，就放到下一个字典中
        if (dic[item] >= val_suport):
            dic_1[item] = dic[item]

    return dic_1


In [43]:
def mining_second(data, dic_before, support, confidence): #挖掘出候选2-项集
  

    dic = {}
    count = len(data)
    count2 = 0
    for data_line in data:
        # 获取元素数量
        count_item = len(data_line)
        # 每两个组合计数
        for i in range(0, count_item - 1):
            #外层循环，控制频繁2-项集中的第一个元素的取值
            for j in range(i + 1, count_item):
                #内层循环，控制频繁2-项集中的第二个元素的取值
                if (data_line[i] in dic_before and data_line[j] in dic_before):

                    count2 += 1
                    tmp = (data_line[i], data_line[j])
                    if (tmp in dic):
                        #上同，使用键值对集合计数，只不过此时元素是二元的元组
                        dic[tmp] += 1
                    else:
                        dic[tmp] = 1
                else:
                    continue
                    #当两个项中有一个不是频繁1-项集，根据剪枝策略，这样组成的项不是频繁2-项集
   
    assert (support >= 0) and (support <= 1), 'suport must be in 0-1'
    assert (confidence >= 0) and (confidence <= 1), 'confidence must be in 0-1'

    dic_2 = {}
    for item in dic:
        count_item0 = dic_before[item[0]]
        count_item1 = dic_before[item[1]]
        # 判断 支持度 和 置信度
        #判断置信度的时候对于一个无序的元组，任何一种方向的规则都有可能，都要进行比较
        if ((dic[item] >= support * count) and (
            (dic[item] >= confidence * count_item0) or (dic[item] >= confidence * count_item1))):
            dic_2[item] = dic[item]

    return dic_2


In [46]:
def mining_third(data, dic_before, support,confidence): # 挖掘出候选3-项集或者4-项集乃至n-项集
    dic_3 = {}
    for item0 in dic_before:#外层循环控制频繁k-1项集中的某一项
        for item1 in dic_before:#内层循环控制频繁k-1项集中的另一项
            if (item0 != item1):
                # print(item0,item1)
                item_len = len(item0)
                equal = True
                tmp_item3 = []
                # 判断前n-1项是否一致
                for i in range(item_len - 1):
                    tmp_item3.append(item0[i])
                    if (item0[i] != item1[i]):
                        equal = False
                        break
                if (equal == True):  #如果两个Fk-1项具有k-2个公共前缀，就按照顺序，将其组合起来
                    minitem = min(item0[-1], item1[-1])
                    maxitem = max(item0[-1], item1[-1])
                    tmp_item3.append(minitem)
                    tmp_item3.append(maxitem)
                    tmp_item3 = tuple(tmp_item3)
                    dic_3[tmp_item3] = 0
                else:
                    continue
    #暴力统计支持度的方法，对于每一个数据项，看每个新找到的k项集是否包含在数据项中
    for data_line in data:
        for item in dic_3:
            is_in = True
            for i in range(len(item)):
                if (item[i] not in data_line):
                    is_in = False
            if (is_in == True):
                dic_3[item] += 1
    assert (support >= 0) and (support <= 1), 'suport must be in 0-1'
    assert (confidence >= 0) and (confidence <= 1), 'coincidence must be in 0-1'
    count = len(data)
    dic_3n = {}
    for item in dic_3:
        count_item0 = dic_before[item[:-1]]
        # 判断 支持度 和 置信度
        if ((dic_3[item] >= support * count) and (dic_3[item] >= confidence * count_item0)):
            dic_3n[item] = dic_3[item]
    return dic_3n

In [45]:


if (__name__ == '__main__'):
    data_row = loadData()

    data_y = preProcessing(data_row, 'y')
    data_n = preProcessing(data_row, 'n')
    
    # 支持度
    support = 0.3
    # 置信度
    confidence = 0.9

    
    r_y = rule_mining(data_y, support, confidence)
    print('vote `y`:\n', r_y)
    

    r_n = rule_mining(data_n, support, confidence)
    print('vote `n`:\n', r_n)
    

   


vote `y`:
 [{(4, 5): 168, (4, 6): 160, (4, 14): 168, (5, 6): 197, (5, 14): 194, (6, 12): 159, (12, 14): 158, (8, 9): 192}, {(4, 5, 6): 156, (4, 5, 14): 160, (4, 6, 14): 151, (5, 6, 14): 180}, {(4, 5, 6, 14): 148}]
vote `n`:
 [{(8, 9): 162, (4, 14): 163, (4, 5): 195, (5, 6): 137, (5, 14): 153}]
